### 爬取所有标签下相关电影一百部

In [1]:
import json
import requests
import os
from tqdm import tqdm_notebook
import operator
import itertools
import numpy as np
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}

In [6]:
#批量获取接口数据的方法
def get(url,headers):
    while True:
        try:
            res = requests.get(url,headers = headers, timeout = 5)
            return json.loads(res.content.decode('utf-8'))
        except requests.exceptions.Timeout:
            time.sleep(1)

In [7]:
#获取电影标签
tags = get('https://movie.douban.com/j/search_tags?type=movie&source=',headers)['tags']

In [ ]:
#每一个标签下获取100部电影信息
import time
V = 100  # 尝试搜索次数，以获得更多结果
id_list = []


search_url = '\
https://movie.douban.com/j/search_subjects?type=movie&tag={}&sort=time&page_limit=20&page_start={}'


for tag in tags:
    print(f"开始搜索标签「{tag}」相关的电影 id")
    for i in tqdm_notebook(range(V)):
        url = search_url.format(tag, 20*i)
        json_content = get(url,headers)
        id_list += [x['id'] for x in json_content['subjects']]

# 去重，一部电影对应多个标签
id_list = list(set(id_list))

# 写入本地文件
with open('movie_id_list.json', 'w',encoding='utf-8') as f:
    f.write(json.dumps({'id': id_list}, indent=4, ensure_ascii=False))

In [ ]:
# 新建文件夹方便整理数据
if not os.path.exists('movies'):
    os.mkdir('movies')

# 读取电影 id 数据
with open('movie_id_list.json') as f:
    id_list = json.load(f)['id']

for movie_id in tqdm_notebook(id_list):
    url = f'https://api.douban.com/v2/movie/subject/{movie_id}?apikey={access_key}'
    json_path = os.path.join('movies', str(movie_id)+'.json')

    # 数据已爬取
    if os.path.exists(json_path):
        with open(json_path) as f:
            t = json.load(f)
            '''
            数据已爬取且没有出错，则跳过本条，例如返回:
            {
                "msg": "rate_limit_exceeded2: 125.70.79.50",
                "code": 112,
                "request": "GET /v2/movie/subject/26611090"
            }
            则是访问超过限制，重新爬取
            '''
            try:
                if t['code'] == 112:
                    pass
            except KeyError:
                continue
    json_content = get(url,headers)
    try:
        # 返回的数据为上面提起的错误，则跳过这条
        if json_content['code'] == 112:
            print('访问超过限制，请稍后再试')
            time.sleep(10)
    except KeyError:
        pass

    with open(json_path, 'w',encoding='utf-8') as f:
        f.write(json.dumps(json_content, indent=4, ensure_ascii=False))

In [ ]:
movies = []
for json_file in tqdm_notebook(os.listdir('movies')):
    json_file_path = os.path.join('movies', json_file)
    with open(json_file_path,encoding='utf-8') as f:
        json_content = json.load(f)
        movies += [json_content]

# 写入文件
with open('movies.json', 'w',encoding='utf-8') as f:
    f.write(json.dumps({'movies': movies}, indent=4, ensure_ascii=False))

###  分析不同标签下的电影

In [2]:
with open('movies.json',encoding = 'utf-8') as f:
    movies = json.load(f)['movies']
len(movies)

4499

#### 不同视频源统计

In [14]:
# 统计视频源和电影类型
# source-> [need_pay 数量，非 need_pay 数量，相应电影类型数量]
movie_types = list(
    set(itertools.chain(*[movie['genres'] for movie in movies])))

# 保存统计结果
source_dict = {}

for movie in movies:
    # 该视频没有视频源，跳过
    try:
        if len(movie['videos']) == 0:
            continue
        # 没有 videos 这个属性也跳过
    except:
        continue
    genres = movie['genres']
    for source in movie['videos']:
        source_name = source['source']['name']
        need_pay = source['need_pay']
        # 如果没有该视频源则都初始化为 0
        if source_name not in source_dict:
            source_dict[source_name] = np.zeros((2+len(movie_types), ))
        # 需要付费的数量
        if need_pay:
            source_dict[source_name][0] += 1
        # 不需要付费的数量
        else:
            source_dict[source_name][1] += 1
        # 统计每个视频源不同类型电影的数量
        for movie_type in genres:
            # 在统计 dict 中的位置
            index = movie_types.index(movie_type)
            source_dict[source_name][2+index] += 1
source_dict

{'爱奇艺视频': array([83.,  2.,  6., 13.,  0.,  2.,  1., 14., 23.,  3.,  0.,  4., 16.,
         2.,  1.,  4.,  0.,  7., 20.,  1., 14., 42.,  0.,  4.,  2., 15.]),
 '优酷视频': array([69.,  2.,  7., 12.,  0.,  1.,  2., 12., 21.,  2.,  0.,  4., 11.,
         2.,  0.,  2.,  0.,  4., 13.,  1., 12., 37.,  1.,  2.,  1., 11.]),
 '芒果TV': array([52.,  3.,  3.,  8.,  0.,  1.,  1.,  9., 19.,  0.,  0.,  1.,  9.,
         0.,  0.,  3.,  0.,  4., 11.,  1.,  9., 28.,  0.,  4.,  1., 12.]),
 '腾讯视频': array([76.,  2.,  8., 11.,  0.,  1.,  1., 15., 23.,  2.,  0.,  4., 12.,
         2.,  0.,  4.,  0.,  4., 14.,  1., 15., 41.,  1.,  3.,  1., 13.]),
 '搜狐视频': array([7., 0., 0., 1., 0., 0., 0., 0., 2., 0., 0., 0., 4., 0., 0., 0., 0.,
        4., 4., 0., 1., 0., 0., 0., 0., 4.]),
 '欢喜首映': array([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1905电影网': array([7., 0., 0., 2., 0., 0., 1., 0., 3., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 4., 

#### 使用条形统计图可视化这些数据

In [ ]:
!pip install plotly

In [5]:
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
trace1 = go.Bar(
    x=list(source_dict.keys()),
    y=np.array(list(source_dict.values()))[:, 0],
    name='需要付费'
)
trace2 = go.Bar(
    x=list(source_dict.keys()),
    y=np.array(list(source_dict.values()))[:, 1],
    name='不需要付费'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='视频源是否需要付费统计')

ModuleNotFoundError: No module named 'plotly'

大一些的在线视频平台一般都很注重版权，付费和非付费对半分。而哔哩哔哩则几乎都是不需要付费的，当然哔哩哔哩的视频数量也不多。同时，像乐视、搜狐这些老牌视频网站，虽然名气很大，反而拥有的电影数目并不多。所以，这些平台的会员一般不值得购买。

#### 使用饼状图可视化 腾讯视频 拥有的电影中不同类型电影的组成

In [ ]:
# 统计标签出现的次数
values = source_dict['腾讯视频'][2:]
# 绘图
trace = go.Pie(labels=movie_types, values=values)
iplot([trace], filename='腾讯视频电影类型百分比')

和统计豆瓣评分 TOP 250 的结果类似，占大头的还是剧情、喜剧、动作等类型的电影。